This is Week 3 Project on Segmenting and Clustering Neighborhoods in Toronto

In [55]:
# importing necessery liberies 
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Fri, 01 May 2020 22:00:05 GMT', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Server': 'ATS/8.0.7', 'Content-Type': 'text/html; charset=UTF-8', 'X-Content-Type-Options': 'nosniff', 'P3P': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-language': 'en', 'Last-Modified': 'Sat, 18 Apr 2020 18:27:33 GMT', 'Content-Encoding': 'gzip', 'Age': '40114', 'X-Cache': 'cp5008 miss, cp5011 hit/76', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Set-Cookie': 'WMF-Last-Access=02-May-2020;Path=/;HttpOnly;secure;Expires=Wed, 03 Jun 2020 00:00:00 GMT, WMF-Last-Access-Global=02-May-2020;Path=/;Domain=.wikipedia.org;HttpOnly;secure;Expires=Wed, 03 Jun 2020 00:00:00 GMT, GeoIP=IN:MH:Pune:18.52:73.85:v4; Path=/; secure; Domain=.wikipedia.org', 'X-Client-IP': '157.47.11.43', 'Cac

# PART 1 ---------------------------------------------Starts...............................

Now lets take therequried data from the wiki

In [9]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    i = tr.find_all('td')
    if i:
        rows.append(i)
rows

[[<td>M1A
  </td>,
  <td>Not assigned
  </td>,
  <td>
  </td>],
 [<td>M2A
  </td>,
  <td>Not assigned
  </td>,
  <td>
  </td>],
 [<td>M3A
  </td>,
  <td>North York
  </td>,
  <td>Parkwoods
  </td>],
 [<td>M4A
  </td>,
  <td>North York
  </td>,
  <td>Victoria Village
  </td>],
 [<td>M5A
  </td>,
  <td>Downtown Toronto
  </td>,
  <td>Regent Park / Harbourfront
  </td>],
 [<td>M6A
  </td>,
  <td>North York
  </td>,
  <td>Lawrence Manor / Lawrence Heights
  </td>],
 [<td>M7A
  </td>,
  <td>Downtown Toronto
  </td>,
  <td>Queen's Park / Ontario Provincial Government
  </td>],
 [<td>M8A
  </td>,
  <td>Not assigned
  </td>,
  <td>
  </td>],
 [<td>M9A
  </td>,
  <td>Etobicoke
  </td>,
  <td>Islington Avenue
  </td>],
 [<td>M1B
  </td>,
  <td>Scarborough
  </td>,
  <td>Malvern / Rouge
  </td>],
 [<td>M2B
  </td>,
  <td>Not assigned
  </td>,
  <td>
  </td>],
 [<td>M3B
  </td>,
  <td>North York
  </td>,
  <td>Don Mills
  </td>],
 [<td>M4B
  </td>,
  <td>East York
  </td>,
  <td>Parkview Hill / Wo

In [20]:
rows[0][0].text.rstrip()

'M1A'

In [16]:
lst = []
for row in rows:
    postalcode = row[0].text.rstrip()
    borough = row[1].text.rstrip()
    neighborhood = row[2].text.rstrip()
    if borough != 'Not assigned':
        if neighborhood == 'Not assigned':
            neighborhood = borough
        lst.append([postalcode, borough, neighborhood])

In [24]:
lst[14]

['M4C', 'East York', 'Woodbine Heights']

In [56]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)
df.shape

(103, 3)

In [22]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [25]:
df = df.groupby('PostalCode').agg({ 'Borough':'first',  'Neighborhood': ', '.join,}).reset_index()

In [26]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [27]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,Regent Park / Harbourfront


In [28]:
df.shape

(103, 3)

# PART 2 -------------------------------------------Starts......................................

In [30]:
dfgeo = pd.read_csv("/home/vaibhav/Desktop/Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [31]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [32]:
geolocator = Nominatim(user_agent="coursera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df2['Latitude'], df2['Longitude'], df2['PostalCode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [33]:
CLIENT_ID = 'AG4OYL0OISHM20FV5QGIDPAKH2HANSOCAVRDD4K33JKJCFPI' # your Foursquare ID
CLIENT_SECRET = 'ODIT1KCQ2GRXXIJFH32UF0P2IS0B0SNVDL0XFMKBN30KS2WT'
VERSION = '20180605' # API version

In [34]:

df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [35]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=AG4OYL0OISHM20FV5QGIDPAKH2HANSOCAVRDD4K33JKJCFPI&client_secret=ODIT1KCQ2GRXXIJFH32UF0P2IS0B0SNVDL0XFMKBN30KS2WT&v=20180605&ll=43.739416399999996,-79.5884369&radius=500&limit=100'

In [36]:
results = requests.get(url).json()
# results

In [37]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:

venues = results['response']['groups'][0]['items']

In [39]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4c633939e1621b8d48842553-0,0,"[{'summary': 'This spot is popular', 'type': '...",4c633939e1621b8d48842553,Subway,"6210 Finch Ave West, Store 103",at Albion Rd.,43.742645,-79.589643,"[{'label': 'display', 'lat': 43.74264512142215...",372,M9V 0A1,CA,Toronto,ON,Canada,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[]
1,e-0-4be58dc4cf200f479154133c-1,0,"[{'summary': 'This spot is popular', 'type': '...",4be58dc4cf200f479154133c,Shoppers Drug Mart,1530 Albion Rd,Albion Mall,43.741685,-79.584487,"[{'label': 'display', 'lat': 43.741685, 'lng':...",405,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[]
2,e-0-4be70e26cf200f47e334153c-2,0,"[{'summary': 'This spot is popular', 'type': '...",4be70e26cf200f47e334153c,Popeyes Louisiana Kitchen,1530 Albion Rd.,at Kipling Ave. (Albion Centre),43.741202,-79.584545,"[{'label': 'display', 'lat': 43.74120165509377...",370,M9V 1B4,CA,Etobicoke,ON,Canada,[1530 Albion Rd. (at Kipling Ave. (Albion Cent...,"[{'id': '4d4ae6fc7a7b7dea34424761', 'name': 'F...",0,[]
3,e-0-5112b872e4b0c0a78d7bcd27-3,0,"[{'summary': 'This spot is popular', 'type': '...",5112b872e4b0c0a78d7bcd27,Sunny Foodmart,1620 Albion road,Albion Road and Finch Ave,43.741840,-79.590561,"[{'label': 'display', 'lat': 43.74184023292111...",319,NaN,CA,Toronto,ON,Canada,"[1620 Albion road (Albion Road and Finch Ave),...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
4,e-0-4cd4738cdfb4a1cd4337535c-4,0,"[{'summary': 'This spot is popular', 'type': '...",4cd4738cdfb4a1cd4337535c,The Beer Store,1530 Albion Rd,Near Finch Ave. W.,43.741694,-79.584373,"[{'label': 'display', 'lat': 43.7416936, 'lng'...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[]
5,e-0-4b04a05bf964a520c45522e3-5,0,"[{'summary': 'This spot is popular', 'type': '...",4b04a05bf964a520c45522e3,Sheriff's No Frills,1530 Albion Rd,at Finch Ave. W.,43.741696,-79.584379,"[{'label': 'display', 'lat': 43.741696, 'lng':...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
6,e-0-4d8ba6910c4e41bdaaf7667f-6,0,"[{'summary': 'This spot is popular', 'type': '...",4d8ba6910c4e41bdaaf7667f,Pizza Pizza,"1530 Albion Road, Unit T25",NaN,43.741569,-79.584489,"[{'label': 'display', 'lat': 43.74156896801906...",397,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
7,e-0-4c1951d6834e2d7f2d3a2a80-7,0,"[{'summary': 'This spot is popular', 'type': '...",4c1951d6834e2d7f2d3a2a80,McDonald's,"1530 Albion Road, Unit F-1",NaN,43.741757,-79.584230,"[{'label': 'display', 'lat': 43.7417571, 'lng'...",427,M9V 1B4,CA,Toronto,ON,Canada,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]
8,e-0-4cd9d00734bb8cfa6576babf-8,0,"[{'summary': 'This spot is popular', 'type': '...",4cd9d00734bb8cfa6576babf,Tim Hortons,6220 Finch Ave West,Albion,43.742015,-79.589690,"[{'label': 'display', 'lat': 43.7420146612893,...",306,M9V 0A1,CA,Etobicoke,ON,Canada,"[6220 Finch Ave West (Albion), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[]


In [40]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,43.742645,-79.589643
1,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
2,Popeyes Louisiana Kitchen,Fried Chicken Joint,43.741202,-79.584545
3,Sunny Foodmart,Grocery Store,43.741840,-79.590561
4,The Beer Store,Beer Store,43.741694,-79.584373


In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:

df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...
M9N,York,Weston,43.706876,-79.518188
M9P,Etobicoke,Westmount,43.696319,-79.532242
M9R,Etobicoke,Kingsview Village / St. Phillips / Martin Grov...,43.688905,-79.554724


# PART 3------------------------------------------------Starts.........................

In [43]:

venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East York
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
North York
York
York
Downtown Toronto
Wes

In [44]:

print(venues.shape)
venues.head()

(2151, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
4,Scarborough,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [45]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,114,114,114,114,114,114
Downtown Toronto,1225,1225,1225,1225,1225,1225
East Toronto,126,126,126,126,126,126
East York,76,76,76,76,76,76
Etobicoke,72,72,72,72,72,72
Mississauga,12,12,12,12,12,12
North York,250,250,250,250,250,250
Scarborough,102,102,102,102,102,102
West Toronto,158,158,158,158,158,158


In [46]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:

grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,...,0.008772,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.008772
1,Downtown Toronto,0.000000,0.000816,0.000816,0.000816,0.001633,0.002449,0.001633,0.015510,0.001633,...,0.009796,0.001633,0.000000,0.003265,0.000000,0.004898,0.000000,0.000000,0.000816,0.005714
2,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.007937,0.000000,0.000000,0.000000,0.023810
3,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.013158
4,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,0.000000
5,Mississauga,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,North York,0.004000,0.004000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000,...,0.000000,0.004000,0.004000,0.008000,0.000000,0.000000,0.000000,0.000000,0.008000,0.000000
7,Scarborough,0.009804,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009804,0.000000,...,0.000000,0.000000,0.000000,0.009804,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,West Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006329,...,0.018987,0.000000,0.000000,0.012658,0.000000,0.006329,0.006329,0.000000,0.000000,0.012658
9,York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000


In [54]:
grouped.shape

(10, 268)

In [48]:

num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
              venue  freq
0       Coffee Shop  0.07
1    Sandwich Place  0.06
2              Park  0.06
3              Café  0.05
4  Sushi Restaurant  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2                Hotel  0.03
3  Japanese Restaurant  0.03
4           Restaurant  0.03


----East Toronto----
                venue  freq
0         Coffee Shop  0.06
1    Greek Restaurant  0.06
2                Café  0.04
3  Italian Restaurant  0.04
4             Brewery  0.04


----East York----
                 venue  freq
0          Coffee Shop  0.07
1                 Bank  0.05
2             Pharmacy  0.04
3  Sporting Goods Shop  0.04
4         Burger Joint  0.04


----Etobicoke----
            venue  freq
0     Pizza Place  0.11
1     Coffee Shop  0.07
2  Sandwich Place  0.07
3             Gym  0.04
4   Grocery Store  0.04


----Mississauga----
                      venue  freq
0             

In [51]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [52]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Cluster Labels,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Pub,Gym
1,Downtown Toronto,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant
2,East Toronto,Cluster Labels,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Pub
3,East York,Coffee Shop,Bank,Park,Pharmacy,Pizza Place,Burger Joint,Sporting Goods Shop,Athletics & Sports,Pet Store,Liquor Store
4,Etobicoke,Cluster Labels,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Gym,Grocery Store,Fast Food Restaurant,Liquor Store,Pet Store
5,Mississauga,Cluster Labels,Hotel,Coffee Shop,Burrito Place,Intersection,Fried Chicken Joint,Mediterranean Restaurant,Middle Eastern Restaurant,Sandwich Place,Gym
6,North York,Cluster Labels,Coffee Shop,Clothing Store,Restaurant,Bank,Pizza Place,Park,Sandwich Place,Japanese Restaurant,Fast Food Restaurant
7,Scarborough,Coffee Shop,Bank,Breakfast Spot,Bakery,Chinese Restaurant,Fast Food Restaurant,Pharmacy,Pizza Place,Indian Restaurant,Intersection
8,West Toronto,Cluster Labels,Café,Bar,Coffee Shop,Italian Restaurant,Restaurant,Pizza Place,Bakery,Breakfast Spot,Gift Shop
9,York,Cluster Labels,Park,Fast Food Restaurant,Spa,Sandwich Place,Brewery,Women's Store,Field,Caribbean Restaurant,Hockey Arena


In [49]:

# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 2, 0, 4, 3, 2, 0, 2, 1], dtype=int32)

In [53]:

merged = grouped

# add clustering labels
merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026316,0.000000,...,Cluster Labels,Coffee Shop,Park,Sandwich Place,Café,Pizza Place,Sushi Restaurant,Dessert Shop,Pub,Gym
1,Downtown Toronto,0.0,0.000816,0.000816,0.000816,0.001633,0.002449,0.001633,0.015510,0.001633,...,Cluster Labels,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Park,Seafood Restaurant
2,East Toronto,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,...,Cluster Labels,Greek Restaurant,Coffee Shop,Brewery,Café,Italian Restaurant,Pizza Place,Ice Cream Shop,Fast Food Restaurant,Pub
3,East York,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Bank,Park,Pharmacy,Pizza Place,Burger Joint,Sporting Goods Shop,Athletics & Sports,Pet Store,Liquor Store
4,Etobicoke,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,...,Cluster Labels,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Gym,Grocery Store,Fast Food Restaurant,Liquor Store,Pet Store
